In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import scipy.io
import random
from collections import defaultdict
import math
import re
import matplotlib.pyplot as plt
from sklearn import preprocessing, linear_model, decomposition
from sklearn.metrics import precision_score, recall_score
import sklearn
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
vitb_mets_X = np.load('Generated_Data/vitb/vitb_mets_X.npy')
vitb_rxns_X = np.load('Generated_Data/vitb/vitb_rxns_X.npy')
vitb_y = np.load('Generated_Data/vitb/vitb_y.npy')

vitb_rxn_list = np.load('Generated_Data/vitb/vitb_rxn_list.npy')
vitb_met_list = np.load('Generated_Data/vitb/vitb_met_list.npy')

In [5]:
len(vitb_mets_X)

593

In [19]:
vitb_types = ['Biotin','Cobalamin','Folate','Niacin','Pantothenate','Pyridoxin','Riboflavin','Thiamin']

In [20]:
np.mean(vitb_y,axis=0)

array([0.33220911, 0.27655987, 0.65767285, 0.55311973, 0.75885329,
       0.56492411, 0.67284992, 0.65261383])

In [21]:
total_size = len(vitb_mets_X)
train_size = int(total_size*0.8)
val_size = int(total_size*0.1)
test_size = total_size - (train_size+val_size)

tr = sorted(random.sample(list(range(total_size)),train_size))
val_n_te = set(range(total_size)).difference(set(tr))
val = sorted(list(random.sample(val_n_te,val_size)))
te = sorted(list(val_n_te.difference(set(val))))

In [22]:
rxn_train_x = vitb_rxns_X[tr]
met_train_x = vitb_mets_X[tr]
train_y = vitb_y[tr]

rxn_val_x = vitb_rxns_X[val]
met_val_x = vitb_mets_X[val]
val_y = vitb_y[val]

rxn_test_x = vitb_rxns_X[te]
met_test_x = vitb_mets_X[te]
test_y = vitb_y[te]

# Vanilla Linear Regression

### Reactions

In [23]:
len(rxn_train_x[0])

5152

In [24]:
lr = linear_model.LinearRegression()

lr.fit(rxn_train_x,train_y)
print('Linear Regression Training MSE: '+str(sklearn.metrics.mean_squared_error(lr.predict(rxn_train_x),train_y)))

Linear Regression Training MSE: 8.600758217437382e-30


In [25]:
print(np.mean(lr.coef_))
np.std(lr.coef_)

0.00024389354265698761


0.007858434642494404

In [26]:
test_preds = lr.predict(rxn_test_x)
test_preds_binary = [[0 if p <= 0.5 else 1 for p in pred] for pred in test_preds]
train_preds = lr.predict(rxn_train_x)
train_preds_binary = [[0 if p <= 0.5 else 1 for p in pred] for pred in train_preds]

print('Linear Regression Test MSE: '+str(sklearn.metrics.mean_squared_error(test_preds,test_y)))

for i in range(8):
    print('Class '+str(i+1)+':') 
    print('Train Precision: '+str(precision_score(np.transpose(train_preds_binary)[i],np.transpose(train_y)[i]))+
          ', Train Recall: '+str(recall_score(np.transpose(train_preds_binary)[i],np.transpose(train_y)[i])))
    print('Test Precision: '+str(precision_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i]))+
          ', Test Recall: '+str(recall_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i])))

Linear Regression Test MSE: 0.06291621397268277
Class 1:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 1.0, Test Recall: 1.0
Class 2:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 0.9333333333333333, Test Recall: 0.8235294117647058
Class 3:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 0.9761904761904762, Test Recall: 0.9534883720930233
Class 4:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 0.9411764705882353, Test Recall: 0.9696969696969697
Class 5:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 0.9534883720930233, Test Recall: 0.9318181818181818
Class 6:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 0.8857142857142857, Test Recall: 0.9117647058823529
Class 7:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 0.9444444444444444, Test Recall: 0.9444444444444444
Class 8:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 0.9444444444444444, Test Recall: 0.918918918918919


In [27]:
# find rxns with highest impact

for i in range(8):
    rxns_n_coefs = sorted(list(zip(vitb_rxn_list,lr.coef_[i])),key=lambda t: abs(t[1]),reverse=True)
    top10 = rxns_n_coefs[:10]
    print('Vitamin: ' +vitb_types[i])
    print(top10)
    for rxn in top10:
        rxn_ind = list(vitb_rxn_list).index(rxn[0])
        #print(all(rxn_train_x[:,rxn_ind]) or not any (rxn_train_x[:,rxn_ind]))

Vitamin: Biotin
[('AOXSr2', 0.11559157272716641), ('AOXSr', 0.1019877804821487), ('CHCOAL', 0.09178265171395561), ('EX_pime(e)', 0.09178265171395561), ('PIMEtr', 0.09178265171395561), ('EACPR2', 0.07647370003327654), ('PMACPME', 0.05523100662254921), ('METS', -0.04840245569787614), ('PYNP2r', -0.04708694427600649), ('THMP', -0.04503631230703907)]
Vitamin: Cobalamin
[('PC6AR', 0.08593012890345726), ('CPC5MT2', 0.0774661340738408), ('R05224', 0.07490698589832454), ('PC6YM', 0.07099775749501779), ('APPLDH', 0.06729137733576222), ('sink_dmbzid', 0.06729137733576222), ('CPC2MT', 0.0629683910720294), ('CPC5MT1', 0.06212138342630658), ('CYRDAR', 0.05464966822718065), ('RE2124C', -0.05106031388652428)]
Vitamin: Folate
[('DHPS2', 0.12653950827565846), ('DHNPA2', 0.11367265642216604), ('HPPK2', 0.11118211721705666), ('DDPGALA', 0.08136762956095167), ('FOLD3', 0.06599301956139923), ('ADCL', 0.05788215174907867), ('ALATA_L', 0.054121607165828894), ('PMDPHT', 0.052894459666854785), ('DHPS', 0.05188

For Biotin synthesis, AOXSr and AOXSr2 (KEGG ID: 2.3.1.47) and CHCOAL (6.2.1.14) are in the synthesis pathways.
For Thiamin, TMK has high negative coefficient -> TMK (thiamine kinase) turns thiamine into thiamine monophosphate. DM_GCALD (sink to let glycol aldehyde out of system) has negative coeff because glycol aldehyde is part of the synthesis pathway.

So we may say that this regression picks up the reactions in the synthesis pathways.

In [28]:
all_constant = []

for i in range(len(rxn_train_x[0])):
    if all(rxn_train_x[:,i]) or not any(rxn_train_x[:,i]):
        all_constant.append(i)
        
print(len(all_constant))
        
lr.coef_[:,all_constant]

178


array([[-1.15559781e-06, -4.51197954e-06,  5.55009141e-06, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-3.54813381e-08, -1.38535284e-07,  1.70409348e-07, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00641924e-07,  3.92951851e-07, -4.83361831e-07, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-4.91278400e-06, -1.91817435e-05,  2.35950604e-05, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-8.85051888e-06, -3.45564517e-05,  4.25071666e-05, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-3.11137308e-06, -1.21482159e-05,  1.49432656e-05, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

### Metabolites

In [29]:
lr = linear_model.LinearRegression()

lr.fit(met_train_x,train_y)
print('Linear Regression Training MSE: '+str(sklearn.metrics.mean_squared_error(lr.predict(met_train_x),train_y)))

Linear Regression Training MSE: 2.558006662499229e-29


In [30]:
print(np.mean(lr.coef_))
np.std(lr.coef_)

0.0005312560429706366


0.02027415583163048

In [31]:
test_preds = lr.predict(met_test_x)
test_preds_binary = [[0 if p <= 0.5 else 1 for p in pred] for pred in test_preds]
train_preds = lr.predict(met_train_x)
train_preds_binary = [[0 if p <= 0.5 else 1 for p in pred] for pred in train_preds]

print('Linear Regression Test MSE: '+str(sklearn.metrics.mean_squared_error(test_preds,test_y)))

for i in range(8):
    print('Class '+str(i+1)+':') 
    print('Train Precision: '+str(precision_score(np.transpose(train_preds_binary)[i],np.transpose(train_y)[i]))+
          ', Train Recall: '+str(recall_score(np.transpose(train_preds_binary)[i],np.transpose(train_y)[i])))
    print('Test Precision: '+str(precision_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i]))+
          ', Test Recall: '+str(recall_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i])))

Linear Regression Test MSE: 0.09029335810277547
Class 1:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 1.0, Test Recall: 1.0
Class 2:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 0.9333333333333333, Test Recall: 0.8235294117647058
Class 3:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 0.9761904761904762, Test Recall: 0.9761904761904762
Class 4:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 0.9117647058823529, Test Recall: 0.9117647058823529
Class 5:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 0.9767441860465116, Test Recall: 0.8936170212765957
Class 6:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 0.9142857142857143, Test Recall: 0.8888888888888888
Class 7:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 0.8611111111111112, Test Recall: 0.9117647058823529
Class 8:
Train Precision: 1.0, Train Recall: 1.0
Test Precision: 0.9444444444444444, Test Recall: 0.8292682926829268


In [32]:
# find mets with highest impact

for i in range(8):
    mets_n_coefs = sorted(list(zip(vitb_met_list,lr.coef_[i])),key=lambda t: abs(t[1]),reverse=True)
    top10 = mets_n_coefs[:10]
    print('Vitamin: ' +vitb_types[i])
    print(top10)
    for met in top10:
        met_ind = list(vitb_met_list).index(met[0])
        #print(all(met_train_x[:,met_ind]) or not any (met_train_x[:,met_ind]))

Vitamin: Biotin
[('pime[c]', 0.19764971193042064), ('pime[e]', 0.19764971193042064), ('pmACP[c]', 0.18070529635205435), ('pmcoa[c]', 0.16871177128962114), ('epACPm[c]', 0.1289863517054425), ('L2aadp[c]', -0.11927368154982089), ('acaadp[c]', -0.11323253025661696), ('aclys[c]', -0.10778037686623948), ('raffin[e]', -0.0958292757987073), ('ala_B[c]', -0.09206009960117646)]
Vitamin: Cobalamin
[('pre6b[c]', 0.15471249666485382), ('pre6a[c]', 0.11835160344864375), ('C02442[c]', -0.09000646670008876), ('hgbam[c]', 0.08600063322994644), ('co1dam[c]', 0.07671598740967758), ('arabinoxyl[e]', -0.0712867166606858), ('arg_L[e]', 0.07085970318254821), ('copre6[c]', 0.06955396662381594), ('codhpre6[c]', 0.0695409038239933), ('pre5[c]', 0.06795153238438476)]
Vitamin: Folate
[('6hmhptpp[c]', 0.22602206584281354), ('6hmhpt[c]', 0.2098000328636892), ('glctnb[c]', 0.15755267688445604), ('cytd[e]', 0.13910405810443885), ('nforglu[c]', 0.11471068345356801), ('core7[e]', 0.11113851496206259), ('4adcho[c]', 0.

For Thiamin, 4mhetz[c](large negative coeff) is a product of thiaminase, hence the large negative coeff?

Also, the y values are AGORA predictions, so the success of linear regression only shows that we get close to their predictions.

# LASSO Regression

### Reactions

In [33]:
len(rxn_train_x[0])

5152

In [34]:
lasso = linear_model.Lasso(alpha=0.01)

lasso.fit(rxn_train_x,train_y)
print('LASSO Regression Training MSE: '+str(sklearn.metrics.mean_squared_error(lasso.predict(rxn_train_x),train_y)))

LASSO Regression Training MSE: 0.02660934078303468


In [35]:
print(np.mean(lasso.coef_))
np.std(lasso.coef_)

0.00021642883423393086


0.009177894461913062

In [36]:
val_preds = lasso.predict(rxn_val_x)
val_preds_binary = [[0 if p <= 0.5 else 1 for p in pred] for pred in val_preds]
train_preds = lasso.predict(rxn_train_x)
train_preds_binary = [[0 if p <= 0.5 else 1 for p in pred] for pred in train_preds]

print('LASSO Regression Val MSE: '+str(sklearn.metrics.mean_squared_error(val_preds,val_y)))

for i in range(8):
    print('Class '+str(i+1)+':') 
    print('Train Precision: '+str(precision_score(np.transpose(train_preds_binary)[i],np.transpose(train_y)[i]))+
          ', Train Recall: '+str(recall_score(np.transpose(train_preds_binary)[i],np.transpose(train_y)[i])))
    print('Val Precision: '+str(precision_score(np.transpose(val_preds_binary)[i],np.transpose(val_y)[i]))+
          ', Val Recall: '+str(recall_score(np.transpose(val_preds_binary)[i],np.transpose(val_y)[i])))

LASSO Regression Val MSE: 0.04712679012549796
Class 1:
Train Precision: 0.993421052631579, Train Recall: 0.9869281045751634
Val Precision: 0.96, Val Recall: 1.0
Class 2:
Train Precision: 0.9927007299270073, Train Recall: 0.9444444444444444
Val Precision: 1.0, Val Recall: 0.8
Class 3:
Train Precision: 0.987012987012987, Train Recall: 0.9806451612903225
Val Precision: 0.95, Val Recall: 0.9743589743589743
Class 4:
Train Precision: 1.0, Train Recall: 0.9705882352941176
Val Precision: 0.9666666666666667, Val Recall: 0.9666666666666667
Class 5:
Train Precision: 0.9805555555555555, Train Recall: 0.9644808743169399
Val Precision: 0.9574468085106383, Val Recall: 0.9375
Class 6:
Train Precision: 0.996268656716418, Train Recall: 0.956989247311828
Val Precision: 0.9375, Val Recall: 0.9090909090909091
Class 7:
Train Precision: 0.9937694704049844, Train Recall: 0.9382352941176471
Val Precision: 1.0, Val Recall: 0.9333333333333333
Class 8:
Train Precision: 0.9807692307692307, Train Recall: 0.95625
Va

In [37]:
test_preds = lasso.predict(rxn_test_x)
test_preds_binary = [[0 if p <= 0.5 else 1 for p in pred] for pred in test_preds]

print('LASSO Regression Test MSE: '+str(sklearn.metrics.mean_squared_error(test_preds,test_y)))

for i in range(8):
    print('Class '+str(i+1)+':') 
    print('Test Precision: '+str(precision_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i]))+
          ', Test Recall: '+str(recall_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i])))

LASSO Regression Test MSE: 0.03154050873985644
Class 1:
Test Precision: 1.0, Test Recall: 1.0
Class 2:
Test Precision: 0.9333333333333333, Test Recall: 0.8235294117647058
Class 3:
Test Precision: 1.0, Test Recall: 1.0
Class 4:
Test Precision: 1.0, Test Recall: 0.9714285714285714
Class 5:
Test Precision: 1.0, Test Recall: 0.9772727272727273
Class 6:
Test Precision: 0.9714285714285714, Test Recall: 0.8717948717948718
Class 7:
Test Precision: 1.0, Test Recall: 0.8780487804878049
Class 8:
Test Precision: 1.0, Test Recall: 0.9


In [38]:
# find rxns with highest impact

for i in range(8):
    rxns_n_coefs = sorted(list(zip(vitb_rxn_list,lasso.coef_[i])),key=lambda t: abs(t[1]),reverse=True)
    top10 = rxns_n_coefs[:10]
    print('Vitamin: ' +vitb_types[i])
    print(top10)
    for rxn in top10:
        rxn_ind = list(vitb_rxn_list).index(rxn[0])
        #print(all(rxn_train_x[:,rxn_ind]) or not any (rxn_train_x[:,rxn_ind]))

Vitamin: Biotin
[('EACPR2', 0.42968480320474645), ('CHCOAL', 0.33666102718151314), ('PMACPME', 0.24273105993401725), ('AOXSr2', 0.2311678464765085), ('EX_pime(e)', 0.12843325985900214), ('AOXSr', 0.024732525631027378), ('PIMEtr', 0.010718389651271398), ('MEVK1c', -0.009488813709789459), ('NAt3', 0.00781312546764339), ('PPAt2r', 0.003372703425057713)]
Vitamin: Cobalamin
[('APPLDH', 0.26425271546825935), ('CPC2MT', 0.1993831354090898), ('CPC5MT2', 0.1715094047949219), ('PC6AR', 0.1421719793407116), ('PC6YM', 0.12143799572170814), ('R05224', 0.11007046068690476), ('CYRDAR', 0.08383287275664425), ('SHCHCC', 0.03214750333471601), ('CBMTHL2', 0.022830655605081313), ('sink_dmbzid', 0.022153854809268864)]
Vitamin: Folate
[('DHPS2', 0.8653715846638236), ('DDPGALA', 0.02829914980174601), ('FOLD3', 0.01728242789933411), ('EX_ppa(e)', 0.015108492890656616), ('PMDPHT', 0.00471752663121331), ('GCCa', 0.004409770429756634), ('ADCL', 0.004396905357962379), ('PANTS', 0.002299709296893714), ('EAR11M12x'

All features variant

In [39]:
all_constant = []

for i in range(len(rxn_train_x[0])):
    if all(rxn_train_x[:,i]) or not any(rxn_train_x[:,i]):
        all_constant.append(i)
        
print(len(all_constant))
print(len(rxn_train_x[0]))
        
print(np.sum(lasso.coef_[:,all_constant],axis=1))
print(np.std(lasso.coef_[:,all_constant],axis=1))

178
5152
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]


Lasso eliminates the effect of all invariant features

### Metabolites

In [40]:
len(met_train_x[0])

2447

In [41]:
lasso = linear_model.Lasso(alpha=0.01)

lasso.fit(met_train_x,train_y)
print('LASSO Regression Training MSE: '+str(sklearn.metrics.mean_squared_error(lasso.predict(met_train_x),train_y)))

LASSO Regression Training MSE: 0.03906649351190376


In [42]:
print(np.mean(lasso.coef_))
np.std(lasso.coef_)

0.00041169163570084553


0.013203602569130576

In [43]:
val_preds = lasso.predict(met_val_x)
val_preds_binary = [[0 if p <= 0.5 else 1 for p in pred] for pred in val_preds]
train_preds = lasso.predict(met_train_x)
train_preds_binary = [[0 if p <= 0.5 else 1 for p in pred] for pred in train_preds]

print('LASSO Regression Val MSE: '+str(sklearn.metrics.mean_squared_error(val_preds,val_y)))

for i in range(8):
    print('Class '+str(i+1)+':') 
    print('Train Precision: '+str(precision_score(np.transpose(train_preds_binary)[i],np.transpose(train_y)[i]))+
          ', Train Recall: '+str(recall_score(np.transpose(train_preds_binary)[i],np.transpose(train_y)[i])))
    print('Val Precision: '+str(precision_score(np.transpose(val_preds_binary)[i],np.transpose(val_y)[i]))+
          ', Val Recall: '+str(recall_score(np.transpose(val_preds_binary)[i],np.transpose(val_y)[i])))

LASSO Regression Val MSE: 0.062013657126327476
Class 1:
Train Precision: 0.993421052631579, Train Recall: 0.9741935483870968
Val Precision: 0.96, Val Recall: 1.0
Class 2:
Train Precision: 0.9854014598540146, Train Recall: 0.8653846153846154
Val Precision: 1.0, Val Recall: 0.7058823529411765
Class 3:
Train Precision: 0.987012987012987, Train Recall: 0.9806451612903225
Val Precision: 0.95, Val Recall: 0.9743589743589743
Class 4:
Train Precision: 1.0, Train Recall: 0.8949152542372881
Val Precision: 0.9666666666666667, Val Recall: 0.8529411764705882
Class 5:
Train Precision: 0.9861111111111112, Train Recall: 0.9806629834254144
Val Precision: 0.9787234042553191, Val Recall: 0.9787234042553191
Class 6:
Train Precision: 0.9552238805970149, Train Recall: 0.9516728624535316
Val Precision: 0.9375, Val Recall: 0.9375
Class 7:
Train Precision: 0.9937694704049844, Train Recall: 0.9327485380116959
Val Precision: 1.0, Val Recall: 0.9333333333333333
Class 8:
Train Precision: 0.9775641025641025, Train 

In [44]:
test_preds = lasso.predict(met_test_x)
test_preds_binary = [[0 if p <= 0.5 else 1 for p in pred] for pred in test_preds]

print('LASSO Regression Test MSE: '+str(sklearn.metrics.mean_squared_error(test_preds,test_y)))

for i in range(8):
    print('Class '+str(i+1)+':') 
    print('Test Precision: '+str(precision_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i]))+
          ', Test Recall: '+str(recall_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i])))

LASSO Regression Test MSE: 0.058288779587913966
Class 1:
Test Precision: 1.0, Test Recall: 1.0
Class 2:
Test Precision: 1.0, Test Recall: 0.7894736842105263
Class 3:
Test Precision: 1.0, Test Recall: 1.0
Class 4:
Test Precision: 1.0, Test Recall: 0.85
Class 5:
Test Precision: 1.0, Test Recall: 0.9772727272727273
Class 6:
Test Precision: 0.8857142857142857, Test Recall: 0.8378378378378378
Class 7:
Test Precision: 1.0, Test Recall: 0.8780487804878049
Class 8:
Test Precision: 0.9722222222222222, Test Recall: 0.8333333333333334


In [45]:
# find mets with highest impact

for i in range(8):
    mets_n_coefs = sorted(list(zip(vitb_met_list,lasso.coef_[i])),key=lambda t: abs(t[1]),reverse=True)
    top10 = mets_n_coefs[:10]
    print('Vitamin: ' +vitb_types[i])
    print(top10)
    for met in top10:
        met_ind = list(vitb_met_list).index(met[0])
        #print(all(met_train_x[:,met_ind]) or not any (met_train_x[:,met_ind]))

Vitamin: Biotin
[('epACPm[c]', 0.6051344148309721), ('pime[c]', 0.4628303008534392), ('pmACP[c]', 0.23258725661737922), ('egACPm[c]', 0.05143933979391657), ('pmcoa[c]', 0.04392616297699127), ('5pmev[c]', -0.011562595428314778), ('gACPm[c]', 0.001702860643686051), ('3hgACPm[c]', 0.0012151912430323332), ('pime[e]', 4.3828445788788125e-15), ('10fthf5glu[c]', 0.0)]
Vitamin: Cobalamin
[('pre6b[c]', 0.6683665720394808), ('pre3a[c]', 0.08953826060553485), ('gam[e]', -0.050510208997825747), ('23ddhb[c]', -0.04659412664896315), ('pser_D[c]', -0.04008879735835773), ('56dthm[c]', 0.03855446051003598), ('hgbam[c]', 0.026851024234385165), ('rib_D[e]', 0.02613353279451607), ('drib[c]', 0.025023126595877753), ('ttc_ggdp[c]', -0.023851157762739734)]
Vitamin: Folate
[('6hmhptpp[c]', 0.8794229559716208), ('2dh3dgal6p[c]', 0.01674439314908353), ('2ahhmd[c]', 0.014957190962627893), ('ppa[e]', 0.0131090602662391), ('sucorn[c]', 0.007326623677913703), ('forglu[c]', 0.0030846402954628075), ('pmcoa[c]', 0.001

# Logistic Regression

### Reactions

In [47]:
LogReg = nn.Sequential(
    nn.Linear(len(vitb_rxns_X[0]),8)
)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(LogReg.parameters(),lr=1e-4,weight_decay=0)

epochs = 15

In [48]:
for i in range(epochs):
    curr_loss = 0
    for k in range(len(rxn_train_x)):
        X = torch.Tensor(rxn_train_x[k])
        y = torch.Tensor(train_y[k])
        
        optimizer.zero_grad()
        
        output = LogReg(X)
        loss = criterion(output, y) + 0.01*torch.norm(LogReg[0].weight.data,1)
        loss.backward()
        optimizer.step()
        
        curr_loss += loss.item()
        
    if i % 5 != 0:
        continue
        
    print('Epoch '+str(i)+', Train MSE: '+ str(curr_loss/len(rxn_train_x)))
        
    with torch.no_grad():
        train_preds = np.transpose(LogReg(torch.Tensor(rxn_train_x)))
        val_preds = LogReg(torch.Tensor(rxn_val_x))
        train_preds_binary = [[0 if p <= 0 else 1 for p in pred] for pred in train_preds]
        val_preds_binary = [[0 if p <= 0 else 1 for p in pred] for pred in val_preds]

        print('Val MSE: '+str(sklearn.metrics.mean_squared_error(val_preds_binary,val_y)))
        
        print('Train Precision: '+str(np.mean([precision_score(train_preds_binary[i],np.transpose(train_y)[i]) for i in range(8)]))+
                  ', Train Recall: '+str(np.mean([recall_score(train_preds_binary[i],np.transpose(train_y)[i]) for i in range(8)])))
        print('Val Precision: '+str(np.mean([precision_score(np.transpose(val_preds_binary)[i],np.transpose(val_y)[i]) for i in range(8)]))+
                  ', Val Recall: '+str(np.mean([recall_score(np.transpose(val_preds_binary)[i],np.transpose(val_y)[i]) for i in range(8)])))

Epoch 0, Train MSE: 3.4602291785212005
Val MSE: 0.21822033898305085
Train Precision: 0.7444510338754325, Train Recall: 0.857989606881298
Val Precision: 0.7276089646169434, Val Recall: 0.8223062068650304
Epoch 5, Train MSE: 4.354703415295242
Val MSE: 0.1313559322033898
Train Precision: 0.9282318732434429, Train Recall: 0.9396089937155229
Val Precision: 0.874782837951056, Val Recall: 0.8566769641054308
Epoch 10, Train MSE: 5.599160766802759
Val MSE: 0.09533898305084745
Train Precision: 0.9613586806230742, Train Recall: 0.97061792559452
Val Precision: 0.9238395049099837, Val Recall: 0.894817847846745


In [49]:
with torch.no_grad():
    test_preds = LogReg(torch.Tensor(rxn_test_x))
    test_preds_binary = [[0 if p <= 0 else 1 for p in pred] for pred in test_preds]

    print('Test MSE: '+str(sklearn.metrics.mean_squared_error(test_preds_binary,test_y)))
    print('Test Precision: '+str([precision_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i]) for i in range(8)])+
              ', Test Recall: '+str([recall_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i]) for i in range(8)]))

Test MSE: 0.09166666666666666
Test Precision: [0.95, 1.0, 0.9523809523809523, 0.8529411764705882, 0.9534883720930233, 0.8571428571428571, 0.9444444444444444, 0.8888888888888888], Test Recall: [0.8636363636363636, 0.7894736842105263, 0.9090909090909091, 0.9354838709677419, 1.0, 0.8823529411764706, 0.8947368421052632, 0.9411764705882353]


In [323]:
print(LogReg[0].weight.data)

tensor([[-0.0069,  0.0782,  0.0218,  ..., -0.0715,  0.2551, -0.0678],
        [-0.0006,  0.0627,  0.0923,  ..., -0.0410,  0.0349, -0.0762],
        [ 0.0045,  0.0525,  0.0226,  ..., -0.0033,  0.0144, -0.0392],
        ...,
        [ 0.0034, -0.0220, -0.0662,  ..., -0.0924, -0.0435, -0.0664],
        [ 0.0203,  0.0092,  0.0397,  ...,  0.0010,  0.0884, -0.0568],
        [ 0.0271, -0.0388,  0.0584,  ...,  0.0691,  0.0679,  0.0518]])


In [325]:
# find rxns with highest impact

for i in range(8):
    rxns_n_coefs = sorted(list(zip(vitb_rxn_list,LogReg[0].weight.data[i])),key=lambda t: abs(t[1]),reverse=True)
    top10 = rxns_n_coefs[:10]
    print('Vitamin: ' +vitb_types[i])
    print(top10)
    for rxn in top10:
        rxn_ind = list(vitb_rxn_list).index(rxn[0])
        #print(all(rxn_train_x[:,rxn_ind]) or not any (rxn_train_x[:,rxn_ind]))

Vitamin: Biotin
[('MALCOACD', tensor(0.3964)), ('3HACPR1', tensor(0.3962)), ('EACPR1', tensor(0.3921)), ('GACPCD', tensor(0.3887)), ('EACPR2', tensor(0.3838)), ('3HACPR2', tensor(0.3803)), ('3OAACPR1', tensor(0.3790)), ('3OAACPR2', tensor(0.3755)), ('AOXSr2', tensor(0.3193)), ('PMACPME', tensor(0.3030))]
Vitamin: Cobalamin
[('PC6YM', tensor(0.3073)), ('CPC5MT2', tensor(0.3047)), ('R05224', tensor(0.3020)), ('PC6AR', tensor(0.2852)), ('APPLDH', tensor(0.2769)), ('CPC5MT1', tensor(0.2737)), ('sink_dmbzid', tensor(0.2706)), ('CPC2MT', tensor(0.2548)), ('CPC4MT', tensor(0.2438)), ('CPC6MT', tensor(0.2385))]
Vitamin: Folate
[('HPPK2', tensor(0.4588)), ('DHPS2', tensor(0.4512)), ('DHNPA2', tensor(0.4492)), ('DDPGALA', tensor(0.3751)), ('ADCL', tensor(0.2801)), ('GCALDD', tensor(0.2628)), ('GLXCL', tensor(0.2541)), ('FE3t4', tensor(0.2497)), ('GLYCLTDxr', tensor(0.2470)), ('r0085', tensor(0.2145))]
Vitamin: Niacin
[('ASPO7', tensor(0.5248)), ('QULNS', tensor(0.4524)), ('ASPO1', tensor(0.4048)

### Metabolites

In [53]:
LogReg = nn.Sequential(
    nn.Linear(len(vitb_mets_X[0]),8)
)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(LogReg.parameters(),lr=1e-4,weight_decay=0)

epochs = 20

In [54]:
for i in range(epochs):
    curr_loss = 0
    for k in range(len(met_train_x)):
        X = torch.Tensor(met_train_x[k])
        y = torch.Tensor(train_y[k])
        
        optimizer.zero_grad()
        
        output = LogReg(X)
        loss = criterion(output, y) + 0.01*torch.norm(LogReg[0].weight.data,1)
        loss.backward()
        optimizer.step()
        
        curr_loss += loss.item()
        
    if i % 5 != 0:
        continue
        
    print('Epoch '+str(i)+', Train MSE: '+ str(curr_loss/len(met_train_x)))
        
    with torch.no_grad():
        train_preds = np.transpose(LogReg(torch.Tensor(met_train_x)))
        val_preds = LogReg(torch.Tensor(met_val_x))
        train_preds_binary = [[0 if p <= 0 else 1 for p in pred] for pred in train_preds]
        val_preds_binary = [[0 if p <= 0 else 1 for p in pred] for pred in val_preds]

        print('Val MSE: '+str(sklearn.metrics.mean_squared_error(val_preds_binary,val_y)))
        
        print('Train Precision: '+str(np.mean([precision_score(train_preds_binary[i],np.transpose(train_y)[i]) for i in range(8)]))+
                  ', Train Recall: '+str(np.mean([recall_score(train_preds_binary[i],np.transpose(train_y)[i]) for i in range(8)])))
        print('Val Precision: '+str(np.mean([precision_score(np.transpose(val_preds_binary)[i],np.transpose(val_y)[i]) for i in range(8)]))+
                  ', Val Recall: '+str(np.mean([recall_score(np.transpose(val_preds_binary)[i],np.transpose(val_y)[i]) for i in range(8)])))

Epoch 0, Train MSE: 2.565445295366054
Val MSE: 0.2457627118644068
Train Precision: 0.6564424066234342, Train Recall: 0.8721351160212238
Val Precision: 0.624762586704076, Val Recall: 0.8391987338519731
Epoch 5, Train MSE: 2.9023507888810043
Val MSE: 0.1461864406779661
Train Precision: 0.8831181705178295, Train Recall: 0.9204934690150965
Val Precision: 0.8395623757891046, Val Recall: 0.86748363912897
Epoch 10, Train MSE: 3.4719913569180774
Val MSE: 0.11864406779661016
Train Precision: 0.9169981817258798, Train Recall: 0.9423087839342286
Val Precision: 0.8801301413568701, Val Recall: 0.8811387677034795
Epoch 15, Train MSE: 4.082319248074721
Val MSE: 0.11440677966101695
Train Precision: 0.9436216734203959, Train Recall: 0.9607899490734884
Val Precision: 0.900001936228665, Val Recall: 0.8764356840554934


In [52]:
with torch.no_grad():
    test_preds = LogReg(torch.Tensor(met_test_x))
    test_preds_binary = [[0 if p <= 0 else 1 for p in pred] for pred in test_preds]

    print('Test MSE: '+str(sklearn.metrics.mean_squared_error(test_preds_binary,test_y)))
    print('Test Precision: '+str([precision_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i]) for i in range(8)])+
              ', Test Recall: '+str([recall_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i]) for i in range(8)]))

Test MSE: 0.10416666666666666
Test Precision: [0.95, 1.0, 0.9047619047619048, 0.7941176470588235, 1.0, 0.8, 0.9444444444444444, 0.8611111111111112], Test Recall: [0.8636363636363636, 0.75, 0.9047619047619048, 0.9642857142857143, 1.0, 0.8484848484848485, 0.8947368421052632, 0.9393939393939394]


In [329]:
print(LogReg[0].weight.data)

tensor([[ 0.0129, -0.0047, -0.0230,  ...,  0.0253,  0.0030, -0.0089],
        [-0.0081,  0.0047, -0.0200,  ...,  0.0542, -0.0306,  0.0033],
        [ 0.0237, -0.0156, -0.0129,  ...,  0.0596, -0.0208, -0.0279],
        ...,
        [ 0.0184, -0.0110, -0.0231,  ...,  0.1151, -0.0129,  0.0033],
        [ 0.0232, -0.0159,  0.0008,  ...,  0.1211, -0.0152, -0.0252],
        [ 0.0153, -0.0270, -0.0179,  ...,  0.1128, -0.0198, -0.0252]])


In [331]:
# find mets with highest impact

for i in range(8):
    mets_n_coefs = sorted(list(zip(vitb_met_list,LogReg[0].weight.data[i])),key=lambda t: abs(t[1]),reverse=True)
    top10 = mets_n_coefs[:10]
    print('Vitamin: ' +vitb_types[i])
    print(top10)
    for met in top10:
        met_ind = list(vitb_met_list).index(met[0])
        #print(all(met_train_x[:,met_ind]) or not any (met_train_x[:,met_ind]))

Vitamin: Biotin
[('malcoam[c]', tensor(0.4647)), ('3ogACPm[c]', tensor(0.4533)), ('gACPm[c]', tensor(0.4473)), ('3opACPm[c]', tensor(0.4453)), ('3hpACPm[c]', tensor(0.4426)), ('3hgACPm[c]', tensor(0.4409)), ('epACPm[c]', tensor(0.4377)), ('egACPm[c]', tensor(0.4336)), ('pmACP[c]', tensor(0.3321)), ('pmACPm[c]', tensor(0.3265))]
Vitamin: Cobalamin
[('pre6b[c]', tensor(0.3397)), ('pre6a[c]', tensor(0.3213)), ('copre5b[c]', tensor(0.3026)), ('hgbam[c]', tensor(0.2883)), ('copre6[c]', tensor(0.2796)), ('codhpre6[c]', tensor(0.2685)), ('pre5[c]', tensor(0.2605)), ('pre4[c]', tensor(0.2581)), ('pre3a[c]', tensor(0.2519)), ('C02442[c]', tensor(-0.2507))]
Vitamin: Folate
[('6hmhptpp[c]', tensor(0.5569)), ('6hmhpt[c]', tensor(0.5380)), ('2dh3dgal6p[c]', tensor(0.3964)), ('5thf[c]', tensor(0.2467)), ('2ahhmd[c]', tensor(0.2370)), ('HC00591[c]', tensor(0.2281)), ('gcald[e]', tensor(0.2218)), ('dhnpt[c]', tensor(0.2147)), ('dhpmp[c]', tensor(0.2109)), ('2ahhmp[c]', tensor(0.2061))]
Vitamin: Niacin

## PCA - How much can we reduce the dimension and retain accuracy?

In [55]:
rxn_train_x_normalized = rxn_train_x.astype('float64')

for i in range(len(rxn_train_x_normalized[0])):
    column = rxn_train_x_normalized[:,i]
    rxn_train_x_normalized[:,i] = (column-np.mean(column))/(np.std(column)+1e-9)

In [56]:
pca = decomposition.PCA(n_components = 128)
pca.fit(rxn_train_x_normalized)
print(pca.explained_variance_ratio_[:10])
print(sum(pca.explained_variance_ratio_))

[0.19103243 0.05646531 0.04672895 0.04335922 0.03191484 0.02603424
 0.0218206  0.01901022 0.01658328 0.01529474]
0.8441534926994904


In [57]:
rxn_train_x_pca = pca.fit_transform(rxn_train_x_normalized)
rxn_train_x_pca.shape

(474, 128)

In [58]:
rxn_test_x_normalized = rxn_test_x.astype('float64')

for i in range(len(rxn_test_x_normalized[0])):
    column = rxn_test_x_normalized[:,i]
    rxn_test_x_normalized[:,i] = (column-np.mean(column))/(np.std(column)+1e-9)

rxn_test_x_pca = pca.transform(rxn_test_x_normalized)

rxn_test_x_pca.shape

(60, 128)

### LR - Reactions

In [443]:
lr_pca = linear_model.LinearRegression()

lr_pca.fit(rxn_train_x_pca,train_y)
print('PCA Linear Regression Training MSE: '+str(sklearn.metrics.mean_squared_error(lr_pca.predict(rxn_train_x_pca),train_y)))

PCA Linear Regression Training MSE: 0.03874068820037886


In [444]:
print(np.mean(lr_pca.coef_))
np.std(lr_pca.coef_)

-0.0004755489606459864


0.007014498542541464

In [445]:
test_preds = lr_pca.predict(rxn_test_x_pca)
test_preds_binary = [[0 if p <= 0.5 else 1 for p in pred] for pred in test_preds]
train_preds = lr_pca.predict(rxn_train_x_pca)
train_preds_binary = [[0 if p <= 0.5 else 1 for p in pred] for pred in train_preds]

print('PCA Linear Regression Test MSE: '+str(sklearn.metrics.mean_squared_error(test_preds,test_y)))

for i in range(8):
    print('Class '+str(i+1)+':') 
    print('Train Precision: '+str(precision_score(np.transpose(train_preds_binary)[i],np.transpose(train_y)[i]))+
          ', Train Recall: '+str(recall_score(np.transpose(train_preds_binary)[i],np.transpose(train_y)[i])))
    print('Test Precision: '+str(precision_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i]))+
          ', Test Recall: '+str(recall_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i])))

PCA Linear Regression Test MSE: 0.06793252576910466
Class 1:
Train Precision: 0.9741935483870968, Train Recall: 0.967948717948718
Test Precision: 0.9047619047619048, Test Recall: 1.0
Class 2:
Train Precision: 1.0, Train Recall: 0.9568345323741008
Test Precision: 1.0, Test Recall: 0.85
Class 3:
Train Precision: 0.9839743589743589, Train Recall: 0.9808306709265175
Test Precision: 0.972972972972973, Test Recall: 1.0
Class 4:
Train Precision: 0.9847328244274809, Train Recall: 0.9626865671641791
Test Precision: 0.9333333333333333, Test Recall: 0.9333333333333333
Class 5:
Train Precision: 0.9888268156424581, Train Recall: 0.9833333333333333
Test Precision: 0.8913043478260869, Test Recall: 1.0
Class 6:
Train Precision: 0.9422382671480144, Train Recall: 0.9490909090909091
Test Precision: 0.8518518518518519, Test Recall: 0.8214285714285714
Class 7:
Train Precision: 0.9811912225705329, Train Recall: 0.9660493827160493
Test Precision: 0.8809523809523809, Test Recall: 0.9487179487179487
Class 8:
T

### LASSO - Reactions

In [59]:
lasso_pca = linear_model.Lasso(alpha=0.01)

lasso_pca.fit(rxn_train_x_pca,train_y)
print('PCA LASSO Regression Training MSE: '+str(sklearn.metrics.mean_squared_error(lasso_pca.predict(rxn_train_x_pca),train_y)))

PCA LASSO Regression Training MSE: 0.04065141886097279


In [60]:
print(np.mean(lasso_pca.coef_))
np.std(lasso_pca.coef_)

-0.0003827744264982707


0.006246642945561395

In [61]:
test_preds = lasso_pca.predict(rxn_test_x_pca)
test_preds_binary = [[0 if p <= 0.5 else 1 for p in pred] for pred in test_preds]
train_preds = lasso_pca.predict(rxn_train_x_pca)
train_preds_binary = [[0 if p <= 0.5 else 1 for p in pred] for pred in train_preds]

print('PCA LASSO Regression Test MSE: '+str(sklearn.metrics.mean_squared_error(test_preds,test_y)))

for i in range(8):
    print('Class '+str(i+1)+':') 
    print('Train Precision: '+str(precision_score(np.transpose(train_preds_binary)[i],np.transpose(train_y)[i]))+
          ', Train Recall: '+str(recall_score(np.transpose(train_preds_binary)[i],np.transpose(train_y)[i])))
    print('Test Precision: '+str(precision_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i]))+
          ', Test Recall: '+str(recall_score(np.transpose(test_preds_binary)[i],np.transpose(test_y)[i])))

PCA LASSO Regression Test MSE: 0.08352304595357915
Class 1:
Train Precision: 0.9605263157894737, Train Recall: 0.9733333333333334
Test Precision: 0.95, Test Recall: 0.8260869565217391
Class 2:
Train Precision: 0.9927007299270073, Train Recall: 0.9645390070921985
Test Precision: 1.0, Test Recall: 0.7894736842105263
Class 3:
Train Precision: 0.9837662337662337, Train Recall: 0.9742765273311897
Test Precision: 0.9285714285714286, Test Recall: 0.8863636363636364
Class 4:
Train Precision: 0.9696969696969697, Train Recall: 0.9552238805970149
Test Precision: 0.7941176470588235, Test Recall: 0.8709677419354839
Class 5:
Train Precision: 0.9861111111111112, Train Recall: 0.9833795013850416
Test Precision: 1.0, Test Recall: 0.9555555555555556
Class 6:
Train Precision: 0.9291044776119403, Train Recall: 0.939622641509434
Test Precision: 0.8, Test Recall: 0.875
Class 7:
Train Precision: 0.9719626168224299, Train Recall: 0.9659442724458205
Test Precision: 0.9444444444444444, Test Recall: 0.8717948717